In [105]:
import scipy.io as sio
import os
import matplotlib.pyplot as plt
import numpy as np

from google.colab import files

In [106]:
machine_data_path = r'/content/data_files/'
try:
    os.mkdir(machine_data_path)
except OSError:
      print ("Creation of the directory %s failed" % machine_data_path)
else:
    print ("Successfully created the directory %s " % machine_data_path)

Creation of the directory /content/data_files/ failed


In [97]:
!wget https://eigenvector.com/wp-content/uploads/2019/06/nir_shootout_2002.mat_.zip

--2021-10-06 06:56:46--  https://eigenvector.com/wp-content/uploads/2019/06/nir_shootout_2002.mat_.zip
Resolving eigenvector.com (eigenvector.com)... 69.163.163.60, 2607:f298:6:a034::eaf:812c
Connecting to eigenvector.com (eigenvector.com)|69.163.163.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5414232 (5.2M) [application/zip]
Saving to: ‘nir_shootout_2002.mat_.zip.1’

nir_shootout_2002.m 100%[===================>]   5.16M  4.97MB/s    in 1.0s    

2021-10-06 06:56:48 (4.97 MB/s) - ‘nir_shootout_2002.mat_.zip.1’ saved [5414232/5414232]



In [98]:
!unzip nir_shootout_2002.mat_.zip -d data_files

Archive:  nir_shootout_2002.mat_.zip
replace data_files/nir_shootout_2002.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data_files/nir_shootout_2002.mat  
replace data_files/__MACOSX/._nir_shootout_2002.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace data_files/__MACOSX/._nir_shootout_2002.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data_files/__MACOSX/._nir_shootout_2002.mat  


In [100]:
# https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [101]:
data_content = loadmat('/content/data_files/nir_shootout_2002.mat')

In [102]:
print(sorted(data_content.keys()))

['__globals__', '__header__', '__version__', 'calibrate_1', 'calibrate_2', 'calibrate_Y', 'test_1', 'test_2', 'test_Y', 'validate_1', 'validate_2', 'validate_Y']


In [103]:
calibrate_1 = data_content['calibrate_1']

In [104]:
calibrate_1

{'author': array([], dtype=float64),
 'axisscale': array([[array([], dtype=float64), array([], dtype=float64)],
        [array([ 600,  602,  604,  606,  608,  610,  612,  614,  616,  618,  620,
         622,  624,  626,  628,  630,  632,  634,  636,  638,  640,  642,
         644,  646,  648,  650,  652,  654,  656,  658,  660,  662,  664,
         666,  668,  670,  672,  674,  676,  678,  680,  682,  684,  686,
         688,  690,  692,  694,  696,  698,  700,  702,  704,  706,  708,
         710,  712,  714,  716,  718,  720,  722,  724,  726,  728,  730,
         732,  734,  736,  738,  740,  742,  744,  746,  748,  750,  752,
         754,  756,  758,  760,  762,  764,  766,  768,  770,  772,  774,
         776,  778,  780,  782,  784,  786,  788,  790,  792,  794,  796,
         798,  800,  802,  804,  806,  808,  810,  812,  814,  816,  818,
         820,  822,  824,  826,  828,  830,  832,  834,  836,  838,  840,
         842,  844,  846,  848,  850,  852,  854,  856,  858,  860

In [93]:
np.shape(calibrate_1['data'][0][0])

TypeError: ignored

In [84]:
np.shape(calibrate_1['axisscale'][0][0][1][0][0])

(650,)

In [87]:
scipy.io.loadmat(calibrate_1, appendmat=True)

TypeError: ignored